In [1]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
path_root = '/home/dmr/DatasetForExperiments/adware'

In [2]:
from keras.preprocessing.image import ImageDataGenerator
batches = ImageDataGenerator().flow_from_directory(directory=path_root, target_size=(64,64), batch_size=10000)

2023-05-03 15:28:02.442918: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Found 107 images belonging to 10 classes.


In [3]:
# Print information about Images 
imgs, labels = next(batches)
imgs.shape # Images with width x length x depth
labels.shape # Lablels with batch_size, number of classes

(107, 10)

In [4]:
import numpy as np
import scipy as sp
from PIL import Image

# Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(imgs/255.,labels, test_size=0.3)

In [5]:
X_train.shape

(74, 64, 64, 3)

In [6]:
X_test.shape

(33, 64, 64, 3)

In [7]:
y_train.shape

(74, 10)

In [8]:
y_test.shape

(33, 10)

In [9]:
# CNN Model

In [10]:
import keras
import tensorflow
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import AUC

In [11]:
num_classes = 10

In [12]:
def malware_model():
    Malware_model = Sequential()
    # Convolutional Layer : 30 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(31, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(64,64,3)))

    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Convolutional Layer : 15 filters, (3 * 3) kernel size
    Malware_model.add(Conv2D(16, (3, 3), activation='relu'))
    
    # Max Pooling Layer : (2 * 2) pool size
    Malware_model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # DropOut Layer : Dropping 25% of neurons.
    Malware_model.add(Dropout(0.25))
    
    # Flatten Layer
    Malware_model.add(Flatten())
    
    # Dense/Fully Connected Layer : 128 Neurons, Relu activation function
    Malware_model.add(Dense(128, activation='relu'))
    
   # Malware_model.add(Dense(64, activation='relu'))
    
    # DropOut Layer : Dropping 50% of neurons.
    Malware_model.add(Dropout(0.5))
    
    # Dense/Fully Connected Layer : 50 Neurons, Softmax activation function
    Malware_model.add(Dense(50, activation='relu'))
    
    # Dense/Fully Connected Layer : num_class Neurons, Softmax activation function
    Malware_model.add(Dense(num_classes, activation='softmax'))
    
    
    Malware_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return Malware_model


    # sparse_categorical_cross_entropy
    # categorical_crossentropy
    

In [13]:
from keras import backend as K

def recall_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_test, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_test, y_pred):
    true_positives = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_test, y_pred):
    precision = precision_m(y_test, y_pred)
    recall = recall_m(y_test, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
Malware_model = malware_model()

2023-05-03 15:28:05.492624: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-05-03 15:28:05.801045: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-03 15:28:05.801063: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-05-03 15:28:05.801446: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 15:28:05.805497: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1999965000 Hz
2023-05-03 15:28:05.8062

In [15]:
Malware_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 31)        868       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 31)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 16)        4480      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [16]:
y_train.shape

(74, 10)

In [17]:
y_train_new = np.argmax(y_train, axis=1)

In [18]:
y_train_new

array([7, 0, 1, 0, 6, 9, 2, 6, 4, 4, 5, 7, 7, 5, 8, 4, 1, 6, 3, 8, 9, 2,
       9, 0, 0, 0, 0, 9, 0, 7, 1, 9, 2, 2, 3, 6, 5, 9, 0, 3, 5, 9, 7, 9,
       2, 2, 7, 3, 4, 1, 0, 5, 8, 7, 5, 3, 6, 8, 6, 4, 4, 3, 7, 3, 1, 2,
       5, 1, 1, 2, 3, 5, 3, 5])

In [19]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                 classes = np.unique(y_train_new),
                                                 y = y_train_new)

class_weights = {l:c for l,c in zip(np.unique(y_train_new), class_weights)}

In [20]:
Malware_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_m,precision_m, recall_m,AUC()])

In [21]:
Malware_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150,  class_weight=class_weights)

2023-05-03 15:28:05.960940: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/150
3/3 [==============================] - 1s 236ms/step - loss: 0.6242 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - auc: 0.4675 - val_loss: 0.3692 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.6471
Epoch 2/150
3/3 [==============================] - 0s 45ms/step - loss: 0.4123 - f1_m: 0.0158 - precision_m: 0.0583 - recall_m: 0.0091 - auc: 0.5498 - val_loss: 0.3340 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5770
Epoch 3/150
3/3 [==============================] - 0s 44ms/step - loss: 0.3971 - f1_m: 0.0154 - precision_m: 0.0486 - recall_m: 0.0091 - auc: 0.5021 - val_loss: 0.3437 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - val_auc: 0.5304
Epoch 4/150
3/3 [==============================] - 0s 40ms/step - loss: 0.3733 - f1_m: 0.0516 - precision_m: 0.1083 - recall_m: 0.0339 - auc: 0.5275 - val_loss: 0.3364 - val_f1_m: 0.0000e+0

Epoch 32/150
3/3 [==============================] - 0s 39ms/step - loss: 0.2121 - f1_m: 0.4011 - precision_m: 0.7538 - recall_m: 0.2758 - auc: 0.9180 - val_loss: 0.2524 - val_f1_m: 0.1579 - val_precision_m: 0.5000 - val_recall_m: 0.0938 - val_auc: 0.8340
Epoch 33/150
3/3 [==============================] - 0s 43ms/step - loss: 0.2375 - f1_m: 0.3127 - precision_m: 0.7301 - recall_m: 0.2000 - auc: 0.8610 - val_loss: 0.2529 - val_f1_m: 0.1538 - val_precision_m: 0.4286 - val_recall_m: 0.0938 - val_auc: 0.8295
Epoch 34/150
3/3 [==============================] - 0s 39ms/step - loss: 0.2119 - f1_m: 0.3227 - precision_m: 0.9153 - recall_m: 0.2000 - auc: 0.9162 - val_loss: 0.2533 - val_f1_m: 0.1538 - val_precision_m: 0.4286 - val_recall_m: 0.0938 - val_auc: 0.8156
Epoch 35/150
3/3 [==============================] - 0s 39ms/step - loss: 0.2001 - f1_m: 0.4630 - precision_m: 0.9193 - recall_m: 0.3102 - auc: 0.9220 - val_loss: 0.2573 - val_f1_m: 0.1538 - val_precision_m: 0.4286 - val_recall_m: 0.093

3/3 [==============================] - 0s 41ms/step - loss: 0.0801 - f1_m: 0.8444 - precision_m: 0.8849 - recall_m: 0.8076 - auc: 0.9920 - val_loss: 0.2422 - val_f1_m: 0.7222 - val_precision_m: 0.7727 - val_recall_m: 0.6875 - val_auc: 0.8705
Epoch 65/150
3/3 [==============================] - 0s 39ms/step - loss: 0.0773 - f1_m: 0.8473 - precision_m: 0.8888 - recall_m: 0.8117 - auc: 0.9951 - val_loss: 0.2445 - val_f1_m: 0.7105 - val_precision_m: 0.7400 - val_recall_m: 0.6875 - val_auc: 0.8677
Epoch 66/150
3/3 [==============================] - 0s 42ms/step - loss: 0.0729 - f1_m: 0.9124 - precision_m: 0.9605 - recall_m: 0.8698 - auc: 0.9945 - val_loss: 0.2539 - val_f1_m: 0.7105 - val_precision_m: 0.7400 - val_recall_m: 0.6875 - val_auc: 0.8613
Epoch 67/150
3/3 [==============================] - 0s 39ms/step - loss: 0.0743 - f1_m: 0.8974 - precision_m: 0.9346 - recall_m: 0.8635 - auc: 0.9930 - val_loss: 0.2492 - val_f1_m: 0.7258 - val_precision_m: 0.7333 - val_recall_m: 0.7187 - val_auc: 

3/3 [==============================] - 0s 40ms/step - loss: 0.0449 - f1_m: 0.9273 - precision_m: 0.9442 - recall_m: 0.9117 - auc: 0.9971 - val_loss: 0.3187 - val_f1_m: 0.7097 - val_precision_m: 0.7167 - val_recall_m: 0.7031 - val_auc: 0.8355
Epoch 97/150
3/3 [==============================] - 0s 39ms/step - loss: 0.0521 - f1_m: 0.9210 - precision_m: 0.9379 - recall_m: 0.9049 - auc: 0.9968 - val_loss: 0.3233 - val_f1_m: 0.7131 - val_precision_m: 0.7241 - val_recall_m: 0.7031 - val_auc: 0.8280
Epoch 98/150
3/3 [==============================] - 0s 38ms/step - loss: 0.0420 - f1_m: 0.9570 - precision_m: 0.9651 - recall_m: 0.9492 - auc: 0.9982 - val_loss: 0.3319 - val_f1_m: 0.7131 - val_precision_m: 0.7241 - val_recall_m: 0.7031 - val_auc: 0.8269
Epoch 99/150
3/3 [==============================] - 0s 39ms/step - loss: 0.0476 - f1_m: 0.8979 - precision_m: 0.9173 - recall_m: 0.8797 - auc: 0.9964 - val_loss: 0.3346 - val_f1_m: 0.7131 - val_precision_m: 0.7241 - val_recall_m: 0.7031 - val_auc: 

3/3 [==============================] - 0s 40ms/step - loss: 0.0345 - f1_m: 0.9221 - precision_m: 0.9221 - recall_m: 0.9221 - auc: 0.9986 - val_loss: 0.3990 - val_f1_m: 0.7097 - val_precision_m: 0.7167 - val_recall_m: 0.7031 - val_auc: 0.7717
Epoch 129/150
3/3 [==============================] - 0s 40ms/step - loss: 0.0304 - f1_m: 0.9603 - precision_m: 0.9731 - recall_m: 0.9479 - auc: 0.9989 - val_loss: 0.4226 - val_f1_m: 0.7222 - val_precision_m: 0.7258 - val_recall_m: 0.7187 - val_auc: 0.7638
Epoch 130/150
3/3 [==============================] - 0s 42ms/step - loss: 0.0329 - f1_m: 0.9300 - precision_m: 0.9388 - recall_m: 0.9221 - auc: 0.9927 - val_loss: 0.4344 - val_f1_m: 0.7381 - val_precision_m: 0.7419 - val_recall_m: 0.7344 - val_auc: 0.7629
Epoch 131/150
3/3 [==============================] - 0s 39ms/step - loss: 0.0290 - f1_m: 0.9484 - precision_m: 0.9567 - recall_m: 0.9404 - auc: 0.9989 - val_loss: 0.4371 - val_f1_m: 0.7258 - val_precision_m: 0.7333 - val_recall_m: 0.7187 - val_au

In [22]:
scores = Malware_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.4739 - f1_m: 0.6905 - precision_m: 0.6935 - recall_m: 0.6875 - auc: 0.7426


In [23]:
loss, accuracy, f1_score, precision, recall = Malware_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.4739 - f1_m: 0.6905 - precision_m: 0.6935 - recall_m: 0.6875 - auc: 0.7426


In [24]:
print('Final CNN accuracy: ', scores[1])

Final CNN accuracy:  0.6904760599136353
